In [90]:
from collections import defaultdict #这是干啥用的？
import jieba
import random
import re

rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}


def is_variable(pattern):
    return pattern.startswith("?") and all(r.isalpha() for r in pattern[1:])

def is_pattern_segment(pattern):
    return pattern.startswith("?*") and all(r.isalpha() for r in pattern[2:])

def is_match(rest,saying):
    if not rest or not saying: return True
    if not all(r.isalpha() for r in rest[0]): return True
    if rest[0] != saying[0]:return False
    return rest[1:],saying[1:]

def match_segmemt(pattern,saying):
    seg_pat,rest=pattern[0],pattern[1:]
    seg_pat=seg_pat.replace("?*","?")
    
    if not rest: return (seg_pat,saying),len(saying)    
    for i,token in enumerate(saying):
        if rest[0]==token and is_match(rest[1],saying[i:]):
            return (seg_pat,saying[:i]),i       
    return (seg_pat,saying),len(saying)


fail=[True,None]#不懂
def segment_match(pattern,saying):
    if not pattern or not saying: return []
    pat=pattern[0] 
    if is_variable(pat):
        return [(pat,saying[0])]+segment_match(pattern[1:],saying[1:])
    elif is_pattern_segment(pat):
        match,place=match_segmemt(pattern,saying)
        return [match] + segment_match(pattern[1:],saying[place:])
    elif pat==saying[0]: return segment_match(pattern[1:],saying[1:])
    else:
        return fail
    
def pattern_to_dict(patterns):

    
    return {k:" ".join(v) if isinstance(v,list) else v for k,v in patterns}

def substitute(rule,parsed_rule):

    if not rule: return []
    
    return [parsed_rule.get(rule[0],rule[0])]+substitute(rule[1:],parsed_rule)

def cut_rules(example):
    if "*" in example:
        x1=example.find("*")
        x2=example.rfind("*")

        if x1==x2:

            part1=" ".join(jieba.cut(example[:x1-1]))+" "
            part2=" "+" ".join(jieba.cut(example[x1+2:]))
            return (part1+example[x1-1:x1+2]+part2).split()

        else:
            part1=" ".join(jieba.cut(example[:x1-1]))+" "
            part2=" "+ " ".join(jieba.cut(example[x2+2:]))
            part3=" "+" ".join(jieba.cut(example[x1+2:x2-1]))+" "
            return (part1+example[x1-1:x1+2]+part3+example[x2-1:x2+2]+part2).split()
    else:
        return " ".join(jieba.cut(example)).split()

def replace(example):
    
    x1=example.find("?")
    x2=example.rfind("?")

    if x1==x2:

        part1=" ".join(jieba.cut(example[:x1]))+" "
        part2=" "+" ".join(jieba.cut(example[x1+2:]))
        return (part1+example[x1:x1+2]+part2).split()

    else:
        part1=" ".join(jieba.cut(example[:x1]))+" "
        part2=" "+ " ".join(jieba.cut(example[x2+2:]))
        part3=" "+" ".join(jieba.cut(example[x1+2:x2]))+" "
        return (part1+example[x1:x1+2]+part3+example[x2:x2+2]+part2).split()
            
def cut_saying(example):
    if "?" in example:
        if example[-1]=="?":
            example=example[:-1]
            return replace(example)+["?"]
        else:
            return replace(example)
        
    else:
        return (" ".join(jieba.cut(example))).split()




In [92]:
def get_new_rule_responses(rule_responses):
    English_rule_responses={}
    Chinese_rule_responses={}
    for rules,responses in rule_responses.items():
        zhmodel = re.compile(u'[\u4e00-\u9fa5]') #判断saying中是否含有中文字符
        match=zhmodel.search(rules)
        if match:
            Chinese_rule_responses[rules]=responses
        else:
            English_rule_responses[rules]=responses
        
    return English_rule_responses,Chinese_rule_responses
a,b=get_new_rule_responses(rule_responses)


            
def get_response(saying,rule_responses):
    zhmodel = re.compile(u'[\u4e00-\u9fa5]') #判断saying中是否含有中文字符
    match=zhmodel.search(saying)
    if match:
        rule_responses=b
        saying=cut_rules(saying)
        for rule,responses in rule_responses.items():
            rule_split=cut_rules(rule)
            if "".join(rule_split[1:-1]) in "".join(saying):
                answer=random.choice(rule_responses[rule])
                answer_split=cut_saying(answer)
                return  " ".join(substitute(answer_split,pattern_to_dict(segment_match(rule_split,saying))))  
                #return segment_match(rule_split,saying)
                
    else:
        rule_responses=a
        saying=saying.split()
        for rule,responses in rule_responses.items():
            rule_split=rule.split()
            #if rule.split()[rule.split().index("?*x")+1:rule.split().index("?*y")] in saying:
            if "".join(rule_split[1:-1]) in "".join(saying):#解决用rule,匹配saying。
                answer=random.choice(rule_responses[rule])
                return  " ".join(substitute(answer.split(),pattern_to_dict(segment_match(rule.split(),saying))))
#             else:
#                 #break
#                 print("Sorry, i can't answer your question, you can ask me something else! ")
        
#get_response("Mom, I feel upset",rule_responses)
get_response("我觉得难受",rule_responses)

'除了 到 这个 ， 你 还有 什么 其他 的 感觉 吗 ？'